In [ ]:
# 전처리

In [25]:
# Colab 업로드
from google.colab import files
import os
import pandas as pd
import numpy as np

print("'202501_clean2.xlsx' 업로드 해주세요")
uploaded = files.upload()
print("현재 디렉토리:", os.listdir())

# 1. 데이터 로드
INPUT_XLSX = "202501_clean2.xlsx"
raw = pd.read_excel(INPUT_XLSX)

print("원본데이터 shape:", raw.shape)


# 2. 금속 컬럼 탐색
import re

patterns = {"Cr": r"(?i)\bCr\b(?!\(VI\))",
            "Co": r"(?i)\bCo\b|Cobalt",
            "Ni": r"(?i)\bNi\b|Nickel",
            "As": r"(?i)\bAs\b|Arsenic",
            "Cd": r"(?i)\bCd\b|Cadmium",
            "Sb": r"(?i)\bSb\b|Antimony",
            "Pb": r"(?i)\bPb\b|Lead"}

metal_map = {}
for metal, pat in patterns.items():
    cand = [c for c in raw.columns if re.search(pat, str(c))]
    if cand:
        metal_map[metal] = cand[0]

print("인식된 금속 컬럼:", metal_map)

# 3. 단위 변환 (ng/m³ → µg/m³)
for m, col in metal_map.items():
    raw[col] = pd.to_numeric(raw[col], errors="coerce") / 1000.0

print("단위 변환 완료 (ng → µg)")

# 4. Cr(VI) 생성
if "Cr" in metal_map:
    raw["Cr(VI)"] = raw[metal_map["Cr"]] / 7.0
    metal_map["Cr(VI)"] = "Cr(VI)"
    print("Cr(VI) 컬럼 생성 완료")

# 5. 결측치 처리
raw = raw.dropna(how="all")  # 행 전체가 NaN이면 제거
print("결측치 제거 후 shape:", raw.shape)

# 6. 극단값 제거 (5–95% 트리밍)
trimmed = {}
for m, col in metal_map.items():
    series = raw[col].dropna()
    ql, qh = series.quantile(0.05), series.quantile(0.95)
    trimmed[m] = series[(series >= ql) & (series <= qh)]

# 7. 트리밍 결과 저장
OUTDIR = "preprocessed"; os.makedirs(OUTDIR, exist_ok=True)

with pd.ExcelWriter(os.path.join(OUTDIR, "preprocessed_data.xlsx")) as writer:
    for m, s in trimmed.items():
        s.to_excel(writer, sheet_name=m, index=False)

print("전처리 완료: preprocessed/preprocessed_data.xlsx 저장됨")


'202501_clean2.xlsx' 업로드 해주세요


Saving 202501_clean2.xlsx to 202501_clean2 (6).xlsx
현재 디렉토리: ['.config', 'preprocessed', '202501_clean2 (5).xlsx', '202501_clean2 (2).xlsx', '202501_clean2 (6).xlsx', 'mc_outputs', '202501_clean2 (4).xlsx', 'fit_outputs', '202501_clean2 (3).xlsx', 'Tx-적합도.xlsx', '202501_clean2 (1).xlsx', '202501_clean2.xlsx', 'sample_data']
원본데이터 shape: (461, 11)
인식된 금속 컬럼: {'Cr': 'Cr(ng/m3)', 'Co': 'Co(ng/m3)', 'Ni': 'Ni(ng/m3)', 'As': 'As(ng/m3)', 'Cd': 'Cd(ng/m3)', 'Sb': 'Sb(ng/m3)', 'Pb': 'Pb(ng/m3)'}
단위 변환 완료 (ng → µg)
Cr(VI) 컬럼 생성 완료
결측치 제거 후 shape: (461, 12)
전처리 완료: preprocessed/preprocessed_data.xlsx 저장됨


In [ ]:
# 최적 분포 피팅

In [ ]:
# 난수 만개 생성

In [ ]:
# LADD/LECR 계산

In [ ]:
# 최종 Cumulative LECR 계산